# File Systems

## FS Structure

Logical Storage Unit, aor a collection of related information

They reside on secondary storage. The kernel driver is the user interface to storage, which maps logical to physical. It provides efficient and convenient access to the disk by allowing data to be stored, located and retrieved efficiently.

Spinning disks provide in place read write and random access
    - io transfers are performed in blocks, or sectors (usually 512 bytes)

File Control Block (FCB) - storage structure consisting of information about a File

Decive Driver - controls the physical device

The device or kernel driver abstracts the drive.. Programmer does not need to know how data is arranged on disk.

The file system is organized into layers.

## Layered File System

1. Application Programs
2. Logical file System
3. File organization Module
4. Basic FIle System
5. I/O control
6. Devices

Device drivers manage Devices and I/O control Layer
- would give command like "read drive1, cylinder72, track2, sector10, into mem location 1060"

Basic File System would be given a command like "retrieve block 123" and translate it to a device driver. It also manages memory buffers and caches (allocating, freeing, and replacement)
    - buffers hold data in transit
    - caches hold frequently used data

File Organization Module understands files, logical addresses, and physical blocks
- it translates the logical plock number to the physical block number
- it manages free space and disk allocation

THe Logical FIle System manages the metadat information. Translates the filename into a file number, file handle, location, by maintaining the control blocks, inodes in UNIX
- directory management, and data protection

Layering is usefull for reducing complexity and redundancy, but adds overhead and can decrease performance. Logical Layers can be implemented by adding any coding methods according to the OS designer.

Many OSs have their own file system. Unif has UFS and FFS, Windows has FAT, FAT32, NTFS as well as floppy. Linux has more than 130 types with extended filesystem

## File System Operations

We have system calls like open(), read(), write(), and close() at the api level but how do we implement them
- on disk and in memory structures.

Boot control block
- contains info needed by the system in order to boot the OS from that volume
- Needed if volume contains OS, usually the first block of volume
- GPT or GUID

Volume Control block (superblock, master file table)
- contains volume details
    - total number of blocks, # of free blocks, # block size, free block pointers or array

Directory structure organizes the files

## File System Implementation

Per file File-Control-Block contains details about the file.
- inode number, permissions, size, dates
- NTFS stores it itnot mater file tables using relational db structures


## In memory file system structures.

Mount Table
- storing system mounts, mount points, file system types

System Wide open file table
- contains a copy of the FCB of each file and other information

Per Process Open File Table 
- contains pointers to appropriate entries in system wide open file table as well as other info


## Open()

Passes filename to the logical file system. The it searches the system wide open file table to determine of its in use.
- If it is in use, then a per process open file table entry is created pointing to the existing system wide open file table
- it it is not in use, then an entry is made in the per process open file table, with a pointer to the entry in the system wide open file table.

## Directory Implementation

Linear list of file names with pointer to the data blocks
- simple to program
- time consuming to execute
    - linear search time
    - could keep ordered alphabetically with linked list or B+ tree

Hash Table - linear list with hash data structure
- Decreases directory search time
- collisions - situations where two file names hash to the same location
- only good entries are fixed size, or use chained overflow method.



## Allcoation Methods

### Contiguous

An allocation method refers to how disk blocks are allocated for files.

Contiguous allocation - each file occupies a set of contiguous blocks
- best performances in most case for a spinngin drive.
- only starting point and number of blocks are required
- finding space for file, knowing file size, external fragmentations, and downtime/online are issues of this method however.

### Event Based systems

Many newer filesystems use a modified contiguous allocation methods 

Event based filesystems allocate disk blocks in extents
- an extens is a continous block of disk space
    - extents are allocated for file allocation
    - a file consists of one or more extents

### Linked
Linked allocation is when each file is a linked list of blocks
- file ends at the NIL pointer
- no external fragmentations
- ech block contains a pointer to the next
- no compaction
- free space management by clustering blocks into groups but increates internal fragmentations
- reliability can be an issue
- localting a block can be time consuming

FAT (File allocation table) variation
- biginning of volume has a table, indexed by block number
- Much like a LL, but faster on disk and cacheable
- new block allocation is simple

### Indexed 

Indexed allocation is when eeach file has its own index block(s) of pointrs to its data blocks.


## Performance

Best method depends on the file access type
- contigous is great for sequential and random access
- Linked is good for sequential but not random
- We declare access type at creation. We select eaither contiguous or Linked

Index is more complex
- single block access could require 2 index block reads before we read data
- clustering can help improve thoughout

For NVM there is no disk overhead, so different algorithms and optimization schemes are needed. Want to reduce cpu utiliation rather than dsik head searching.


## Free Space Management

The FS maintains a free-space list to track available blocks/clusters

We have a bit vector or bit map to keep track of which blocks are available. 1 for free 0 for occupied.
The block number calculation is (number of bits per word)*(number of 0-value words) + offset of first bit

### Linked Free Space on Disk
- Cannot get contiguous space easily
- no waste of space however
- no need to traverse the entire list if the number of free blocks are recorded.

Grouping
- modifying linked list to store the address of the next n-1 free blocks in the first free block, plus a pinter to the next block that contains three block pointers
- Counting, Because space is always being contiguously used and freed, with contiguous allocation, extents, or clustering
    - We keep addresses addresses of the first free block and count the following free blocks
    - the free space list then has entries containing addresses and counts.

## SDD trim

Tells the sdd that specific areas have blocks no longer in use.Data has been deleted form a document maybe. Active garbage collection will free that space in the memory during an idle time. If we didn't have that, then the SDD would have to erase the data right before writing back to it, which would take more time

Trimming Unused Blocks

Spinning Drived overwrite inplace, so only need to free list

Blocks are not treated specially when freeing
- it keeps its data but without any file pointers to it, until overwritten

Storage devieces not allowing overwrite(like NVM) suffer with this algorithm
- it must be erased efore writter, and erasing large chunks is slow
- TRIM is newer and allows the file system to inform the NVM ssd that a page is free

### Efficiency and Performance

Efficiency Depends on
- disk allocation and directory algorithm
- Types of data kept in files directory entry
- Pre allocation or as needed allocation of metadata structures
- fixed size or varying size data structures

Performance Depends on
- keeping metaata and data close together
- Buffer cache - separates ram for frequently used blocks
- Synchronous writes sometimes requested by apps or are needed by os
    - no buffereing or cacheing - writes must hit before acknowledgement
    - asynchronous writes are more common, bufferable, and faster (write through no allocate)

- Free Behind
    - Removes a page from the buffer as soon as the next page is requested. Previous pages are not likely to be requested
- Read ahead
    - A requested page and several subsequent pages are read and cached, these pages are likely to be requested after the current page is processed



## Page Cache

Caches pages rather than disk blocks, using virtual memory techniques and addresses
- memory mapped IO uses page Caches
- routine IO though the file system uses the buffer(disk) Cache

## Buffer Cache

The buffer cache allows us to cache both the memory mapped pages and the ordinary file system IO to avoid double caching.

_Which page gets priotiy and what replacement algorithms do we use_


## Recovery

Consistecy Checking compares data in the directory stucture with blocks on the disk, and tries to fix inconsistencies.
- can be slow and sometimes fails

User system programs to back up data from one disk to another, and then later used to recover the data

## Log Structured File Systems

Record each metadata update to the filesystem as a transaction

All transactions are written to a log
- considered commited once it is written to teh log (sequentially)
- sometimes in seperate disk or seperate section of disk
- FS may not be updated yet

The transactionsa are sequentially written to the log  asycnronously to the fs structures when the computer has spare time.
    - when the fs system structures are modified, the transaction is removed form the log.

If the fs crashes, then the remaining transactions in the log must still be performed when the computer comes back up

This means faster recovery from crash , and removes the chance of inconsistency of metadata

